In [104]:
import cPickle as cp
import numpy as np
import matplotlib.pyplot as plt
from __future__ import division

X, y = cp.load(open('data/voting-full.cPickle', 'rb'))

N, D = X.shape
N_train = int(0.8 * N)
N_test = N - N_train
X_train = X[:N_train]
y_train = y[:N_train]
X_test = X[N_train:]
y_test = y[N_train:]

report_string = "{}\nTrain: {:.5f}\nTest:  {:.5f}"

print X[1]
print X[1].shape

[ 1.  0.  1.  0.  0.  0.  1.  1.  1.  0.  0.  0.  0.  0.  1.  2.]
(16,)


In [138]:
from scipy.stats import norm
from math import log

class NBC:
    def __init__(self, feature_types, num_classes):
        self.feature_types = feature_types
        self.num_features = len(feature_types)
        self.num_classes = num_classes
        self.class_counts = None
        self.class_params = None
        
    
    def fit(self, X, y):
        if (self.class_counts != None):
            raise Exception("Already fitted")
        if (X.shape[1] != self.num_features):
            raise Exception("Wrong number of features")
            
        self.class_counts = np.array([np.count_nonzero(y == i) for i in range(0, self.num_classes)]) / y.shape[0]
        self.class_params = np.array([
            self._compute_params(
                self.feature_types[feature], 
                [X[i] for i in range(0,X.shape[0]) if y[i] == class_])
            for feature in np.arange(0, len(self.feature_types))
            for class_ in np.arange(0, self.num_classes)]).reshape(self.num_features, self.num_classes)
                    
    def _compute_params(self, type_, Xj):
        if type_ == 'b':
            return lambda x: np.mean(Xj) if x == 1 else 1-np.mean(Xj)
        elif type_ == 'r':
            return norm(np.mean(Xj), np.std(Xj)).pdf
        
    def predict(self, X):
        foo = [log(self.class_counts[class_]) + sum([log(self.class_params[feature,class_](X[:,feature])) for feature in range(0, self.num_features)]) 
             for class_ in range(0, self.num_classes)]
        print foo
    
nbc = NBC(['b','b','b','b','b','b','b','b','b','b','b','b','b','b','b','r'], 2)
nbc.fit(X_train, y_train)
print nbc.predict(X_test)
    
print nbc.class_counts
print nbc.class_params

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [91]:
from sklearn.naive_bayes import MultinomialNB 

for nbc in [NBC(None, 2), MultinomialNB()]:
    nbc.fit(X_train, y_train)
    y_hat = nbc.predict(X_test)
    test_accuracy = np.mean(y_hat == y_test)
    print test_accuracy

[0.39080459770114945, 0.6091954022988506]
0.632183908046
0.931034482759


In [96]:
print X_train.reduce(lambda x, y:  x + y)

AttributeError: 'numpy.ndarray' object has no attribute 'reduce'